In [6]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt

In [7]:
%load_ext autoreload
%autoreload 2

## Task 1: What is the relationship between hours worked and exam results?

Some might say that the more hous you log, the better your exam scores are. Is that really true?

In [8]:
data = pd.read_csv('regression_data/mission1.csv')

In [9]:
# TODO: Implement Linear Regression from scratch
class LinearRegression:
    def __init__(self):
        pass

    def fit(self, X, y):
        pass

    def predict(self, X):
        pass


In [10]:
# TODO: Visualize the data
# TODO: Implement Linear Regression
# TODO: Estimate the coefficients (effect of studying) through Linear Regression
# TODO: Visualize the regression line


## 🧠 Task 2: Logistic Regression for Binary Classification

### Mission
1. From your Linear Regression model, make a Logistic Regressio model that performs binary classification
2. Look into how you can use feature engineering to improve the accuracy
3. Solve it using a Decision Tree instead

**Performance goal**: 0.88 accuracy on the test set

In [11]:
data = pd.read_csv('regression_data/mission2.csv')

In [12]:
# TODO: Implement Logistic Regression from scratch
class LogisticRegression:
    def __init__(self):
        pass

    def fit(self, X, y):
        pass

    def predict(self, X):
        pass

In [13]:
# TODO: Visualize the data. Is it linearly separable?
# TODO: What can be done to make it linearly separable?